## Questionamentos a serem levantados (1 etapa)

ordenamento de dataset e importante para analise?

Quais as hipoteses possiveis?

Quais as colunas sao desnecessarias?

Tratamento de outliers e valores NaN


In [3]:
import pandas as pd
import numpy as np

In [2]:
data=pd. read_csv("data.csv")
data.head

<bound method NDFrame.head of                                address                 district  area  \
0                           Rua Herval               Belenzinho    21   
1                   Avenida São Miguel             Vila Marieta    15   
2                     Rua Oscar Freire                Pinheiros    18   
3                     Rua Júlio Sayago                  Vila Ré    56   
4                   Rua Barata Ribeiro               Bela Vista    19   
...                                ...                      ...   ...   
11652          Avenida Adolfo Pinheiro              Santo Amaro    94   
11653  Rua Professor Alexandre Correia     Jardim Vitória Régia   350   
11654                 Rua Abílio Borin         Jardim Caravelas    84   
11655                Rua Forte William  Jardim Fonte do Morumbi    76   
11656              Rua Alexandre Dumas              Santo Amaro   110   

       bedrooms  garage                type   rent  total  
0             1       0     Studi